In [ ]:
# collect data

In [ ]:
# fine-tune yolo nas model to work with custom dataset

In [ ]:
# f = pd.DataFrame(violence)
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
%%capture
!pip install super-gradients==3.1.0
!pip install imutils
!pip install roboflow
!pip install pytube --upgrade

In [4]:
from super_gradients.training import models

In [2]:
from super_gradients.training import Trainer

CHECKPOINT_DIR = '/content/checkpoints'
trainer = Trainer(experiment_name='/content/my_first_yolo_nas_l_run', ckpt_root_dir=CHECKPOINT_DIR)

[2023-05-13 04:54:52] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-05-13 04:54:59] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-05-13 04:54:59] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-05-13 04:54:59] WARNING - export.py - Failed to import pytorch_quantization
[2023-05-13 04:54:59] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization


In [7]:
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val

In [8]:
dataset_params = {
    'data_dir':'/content/drive/MyDrive/maruti_suzuki',
    'train_images_dir':'/content/drive/MyDrive/maruti_suzuki/images/train',
    'train_labels_dir':'/content/drive/MyDrive/maruti_suzuki/labels/train',
    'val_images_dir':'/content/drive/MyDrive/maruti_suzuki/images/val',
    'val_labels_dir':'/content/drive/MyDrive/maruti_suzuki/labels/val',
    'classes': ['maruti_brand_name', 'maruti_logo']
}

In [5]:
from google.colab import drive
drive.mount('/content/drive')

In [9]:
from IPython.display import clear_output

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

clear_output()

Caching annotations: 100%|██████████| 20/20 [00:00<00:00, 320.35it/s]


# Instantiate the model

In [10]:
from super_gradients.training import models
model = models.get('yolo_nas_l', 
                   num_classes=len(dataset_params['classes']), 
                   pretrained_weights="coco"
                   )

[2023-05-13 05:43:04] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.


# 


max_epochs - Max number of training epochs

loss - the loss function you want to use

optimizer - Optimizer you will be using

train_metrics_list - Metrics to log during training

valid_metrics_list - Metrics to log during training

metric_to_watch - metric which the model checkpoint will be saved according to

You can choose from a variety of optimizer's such as: Adam, AdamW, SGD, Lion, or RMSProps. If you choose to change the defualt parameters of these optimizrs you pass them into optimizer_params.

🧑🏾‍🔬 Integrations with experiment monitoring tools
SuperGradients has native integrations with Tensorboard, Weights and Biases, ClearML, and DagsHub.

If your favorite monitoring tool is not supported by SuperGradients, you can simply implement a class inheriting from BaseSGLogger that you will then pass to the training parameters.

If you're interested in monitoring experiments, you can learn more in the docs.

🪄 SuperGradients offers a number of training tricks right out of the box, such as:
Exponential moving average
Zero weight decay on bias and batch normalizatiom
Weight averaging
Batch accumulation
Precise BatchNorm

In [8]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    # ENABLING SILENT MODE
    'silent_mode': True,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    # ONLY TRAINING FOR 10 EPOCHS FOR THIS EXAMPLE NOTEBOOK
    "max_epochs": 20,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        # NOTE: num_classes needs to be defined here
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            # NOTE: num_classes needs to be defined here
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

# Training the model

In [9]:
trainer.train(model=model, 
              training_params=train_params, 
              train_loader=train_data, 
              valid_loader=val_data)

[2023-05-13 04:58:41] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}
[2023-05-13 04:58:48] INFO - utils.py - NumExpr defaulting to 2 threads.


The console stream is now moved to /content/my_first_yolo_nas_l_run/console_May13_04_58_48.txt


[2023-05-13 04:58:56] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               1          (1 available on the machine)
    - Dataset size:                 275        (len(train_set))
    - Batch size per GPU:           16         (batch_size)
    - Batch Accumulate:             1          (batch_accumulate)
    - Total batch size:             16         (num_gpus * batch_size)
    - Effective Batch size:         16         (num_gpus * batch_size * batch_accumulate)
    - Iterations per epoch:         17         (len(train_loader))
    - Gradient updates per epoch:   17         (len(train_loader) / batch_accumulate)

[2023-05-13 04:59:51] INFO - base_sg_logger.py - Checkpoint saved in /content/my_first_yolo_nas_l_run/ckpt_best.pth
[2023-05-13 04:59:51] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.006859308574348688
[2023-05-13 05:00:39] INFO - base_sg_logger.py - Checkpoint saved in /

SUMMARY OF EPOCH 20
├── Training
│   ├── Ppyoloeloss/loss = 1.4966
│   │   ├── Best until now = 1.5158 (↘ -0.0191)
│   │   └── Epoch N-1      = 1.5158 (↘ -0.0191)
│   ├── Ppyoloeloss/loss_cls = 0.724
│   │   ├── Best until now = 0.7414 (↘ -0.0173)
│   │   └── Epoch N-1      = 0.7414 (↘ -0.0173)
│   ├── Ppyoloeloss/loss_dfl = 0.8723
│   │   ├── Best until now = 0.8669 (↗ 0.0054)
│   │   └── Epoch N-1      = 0.8669 (↗ 0.0054)
│   └── Ppyoloeloss/loss_iou = 0.1346
│       ├── Best until now = 0.1364 (↘ -0.0018)
│       └── Epoch N-1      = 0.1364 (↘ -0.0018)
└── Validation
    ├── F1@0.50 = 0.1336
    │   ├── Best until now = 0.142  (↘ -0.0084)
    │   └── Epoch N-1      = 0.1167 (↗ 0.0169)
    ├── Map@0.50 = 0.9287
    │   ├── Best until now = 0.945  (↘ -0.0162)
    │   └── Epoch N-1      = 0.945  (↘ -0.0162)
    ├── Ppyoloeloss/loss = 1.3332
    │   ├── Best until now = 1.3444 (↘ -0.0112)
    │   └── Epoch N-1      = 1.3444 (↘ -0.0112)
    ├── Ppyoloeloss/loss_cls = 0.679
    │   ├── Be

Now that training is complete, you need to get the best trained model.

You used checkpoint averaging so the following code will use weights averaged across training runs.

If you want to use the best weights, or weights from the last epoch you'd use one of the following in the code below:

best weights: checkpoint_path = checkpoints/my_first_yolonas_run/ckpt_best.pth

last weights: checkpoint_path = checkpoints/my_first_yolonas_run/ckpt_latest.pth

In [11]:
best_model = models.get('yolo_nas_l',
                        num_classes=len(dataset_params['classes']),
                        checkpoint_path="/content/my_first_yolo_nas_l_run/average_model.pth")

[2023-05-13 05:43:22] INFO - checkpoint_utils.py - Successfully loaded model weights from /content/my_first_yolo_nas_l_run/average_model.pth EMA checkpoint.


In [ ]:
# trainer.test(model=best_model,
#             test_metrics_list=DetectionMetrics_050(score_thres=0.1, 
#                                                    top_k_predictions=300, 
#                                                    num_cls=len(dataset_params['classes']), 
#                                                    normalize_targets=True, 
#                                                    post_prediction_callback=PPYoloEPostPredictionCallback(score_threshold=0.01, 
#                                                                                                           nms_top_k=1000, 
#                                                                                                           max_predictions=300,                                                                              
#                                                                                                           nms_threshold=0.7)
#                                                   ))

In [ ]:
# predicting an image
img_url = '/content/drive/MyDrive/maruti_suzuki/images/train/Maruti-Suzuki_100_jpg.rf.565f66fdc611c2c24185f822c50b5a2b.jpg'
best_model.predict(img_url, conf=0.39).show()

In [ ]:
img_url1 = '/content/drive/MyDrive/maruti_suzuki/images/train/image24.jpeg'
best_model.predict(img_url1).show()

# passing a video for testing

In [ ]:
import torch
device = 0 if torch.cuda.is_available() else "cpu"
input_video_path = "/Im7pyHj-AUs.mp4" 
output_video_path = "yolonas_l_detections.mp4"
best_model.to(device).predict(input_video_path).save(output_video_path)